# Main analysis on api collected data to follow

In [92]:
# import libraries

import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
import json
from pathlib import Path
import numpy as np
import datetime
import dateutil.parser
import urllib.request
from dotenv import load_dotenv

%matplotlib inline


In [93]:
#pn.extension('plotly')

In [94]:
import hvplot.pandas

In [95]:
# Read the API keys
load_dotenv()

True

In [96]:
# Define Algorithms, Assets to evaluate
# this is hard coded for first MVP
eval_pos_list = ['FAANG','Resi_RE','GlobalArtIndex']
benchmk_pos_list = ['SP500','BondAgg','BTCUSD','XAUUSD']
calc_list = ['returns','cum_returns','std','sharpe']
plot_list = ['price','returns','cum_returns','std','sharpe']


In [97]:
# Create latest up to date dataset for each position reviewed:
# A historical dataset is created by a separate script that pulls, massages the data and creates a csv.
# This data is then pulled into a df and then updated with latest market information using appropriate API
# get request. This way historical data doesnt need to be retrieved with repetitive API gets, but the data 
# can be kept up to date with latest info.

# #each should return, cleaned properly and format
#def BondAgg_latest_data
#
#date adj close price
#
#def BTCUSD_latest_data
#
#def XAUUSD_latest_data

## Fetch ART Data ##
art_file_path = Path("../data_imports/Artprice_Indexes_Quarterly_data_Base100_January1998.csv")
art_index = pd.read_csv(art_file_path, encoding='utf-8', parse_dates=True, infer_datetime_format=True)
art_index.rename(columns=({"Unnamed: 0" : "Date"}), inplace=True)
art_index.dropna()
art_index.set_index("Date", drop=True, inplace=True)
#art_index.tail()

In [98]:
## Fetch SP500 Data ##
ms_api_key = os.getenv("MARKETSTACK_API_KEY")
ms_api_key_string = "&access_key=" + ms_api_key 
print(type(ms_api_key))
ms_request_url = "https://api.marketstack.com/v1/eod?symbols=GSPC.INDX"
ms_request_url = ms_request_url + ms_api_key_string + "&limit=3000" 
# Execute get request with API key
ms_response_data = requests.get(ms_request_url)
ms_data = ms_response_data.json()
#print(json.dumps(data, indent=4))
sp_list = ms_data["data"]
# Pull json outout into dataframe and update the adjusted closing prices into columns
column_list = ['date', 'sp500_adj_closing']
sp500_df = pd.DataFrame(columns=column_list)
for dict in sp_list: 
    for list in dict: 
        if list == "date":
            temp_date = str(dateutil.parser.parse(dict[list]).date())
            #print(temp_date)
        elif list == "adj_close":
            temp_close = dict[list] 
            
    new_row = {'date':temp_date, 'sp500_adj_closing':temp_close}
    sp500_df = sp500_df.append(new_row, ignore_index=True)                 
    
sp500_df["date"] = pd.to_datetime(sp500_df["date"])
sp500_df.set_index("date", drop=True, inplace=True)
sp500_df.rename(columns = {"sp500_adj_closing" : "sp500_benchmark"}, inplace=True)
sp500_df.head()

<class 'str'>


,sp500_benchmark
date,
2020-11-04,3443.4399
2020-11-03,3369.1599
2020-11-02,3310.2400
2020-10-30,3269.9600
2020-10-29,3310.1101


In [113]:
## Fetch crypto Data ##
api_key = os.getenv("NOMICS_API_KEY")
print(type(api_key))
api_key_string = "key=" + api_key
request_url = "https://api.coindesk.com/v1/bpi/historical/close.json?start=2012-01-01&end=2020-10-05"
# Execute get request
response_data = requests.get(request_url)
data = response_data.json()
#print(json.dumps(data, indent=4))
bpi_list = data["bpi"]
#print(json.dumps(bpi_list, indent=4))
bpi_df = pd.DataFrame.from_dict(bpi_list, orient='index',
                       columns=['bpi_closing'])
#bpi_df.columns = ['bpi_closing']
#bpi_df.set_index(bpi_df['date'],inplace=True)
bpi_df.head()

<class 'str'>


,bpi_closing
2012-01-01,5.2677
2012-01-02,5.2168
2012-01-03,4.8808
2012-01-04,5.5738
2012-01-05,6.9476


In [111]:
# Combine datasets into one daily adjusted closing USD price dataframe

# first select art index of choice and scale up price by 100
gblart_index_df['GlobalArtIndex'] = (art_index.loc[:,'Global Index (USD)'])*100

benchmk_pos_list = ['SP500','GlobalArtIndex']

test_price_df = pd.DataFrame(columns=benchmk_pos_list)
test_price_df = sp500_df

test_price_df =  test_price_df.merge(gblart_index_df, 
                                                     how='left',
                                                     left_index=True,
                                                     right_index=True
                                                    )

test_returns_df = test_price_df.pct_change() #
test_std_df = test_price_df.std()
test_std_df



#sp500_daily_returns = sp500_df.pct_change().dropna()
#sp500_daily_returns.head()

sp500_benchmark     618.093844
GlobalArtIndex     3694.871980
dtype: float64

In [82]:
#gblart_returns_df = pd.DataFrame(columns=['GlobalArtIndex'])
#gblart_returns_df['GlobalArtIndex'] = art_index.loc[:,'Global Index (USD)'].pct_change().dropna()
#test_combo_plot = gblart_returns_df.hvplot.line(title="test combo Returns", ylabel="Daily Returns", frame_width=900, frame_height=450, rot=90)
#test_combo_plot
#gblart_index_df = pd.DataFrame(columns=['GlobalArtIndex'])



,GlobalArtIndex
Date,
1998-01-01,100
1998-04-01,107
1998-07-01,106
1998-10-01,96
1999-01-01,104


In [80]:
#returns_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#cum_returns_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#std_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)
#sharpe_benchmk_10yr_df = pd.DataFrame(columns=benchmk_pos_list)

#returns_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#cum_returns_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#std_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)
#sharpe_eval_10yr_df = pd.DataFrame(columns=eval_pos_list)

#returns_eval_10yr_df['GlobalArtIndex'] = art_index.loc[:,'Global Index (USD)'].pct_change().dropna()
#returns_eval_10yr_df.head()

# test merge trying zero fill NaN locations to help with line plot below -- spoiler alert -- doesnt work
#test_combo_df = sp500_daily_returns.merge(gblart_returns_df,how='left',left_index=True,right_index=True).fillna(0)
# long head here to verify that its joining where there's valid art data
#test_combo_df.head(100)


,sp500_benchmark,GlobalArtIndex
date,,
2020-11-03,-0.021571,0.000000
2020-11-02,-0.017488,0.000000
2020-10-30,-0.012168,0.000000
2020-10-29,0.012278,0.000000
2020-10-28,-0.011806,0.000000
2020-10-27,0.036579,0.000000
2020-10-26,0.003035,0.000000
2020-10-23,0.018942,0.000000
2020-10-22,-0.003434,0.000000


In [81]:
#test_combo_plot = test_combo_df.hvplot.line(title="test combo ", ylabel="Returns", frame_width=900, frame_height=450, rot=90)
#test_combo_plot

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [8]:
# Define Panel Visualization Functions
# Number of plot functions right now = 5 calcs/position * 2timeframes * 6 positions = 60!!!!


#def 
#art_index_std_bar = px.bar(art_index_std, title="Art Price STD 1998-2020")
#art_index_std_bar

In [2]:
# Create Panel Dashboard

